In [62]:
#imports
import pandas as pd
from scipy.stats import mannwhitneyu
import statsmodels.api as sm
import statsmodels.formula.api as smf
import ace_tools_open as tools

In [63]:
# Load the data
file_path = "lrv-con.hbo.v3.allchannel.csv"
#file_path= "lrv-pcon.hbo.v7.allchannel.csv"
#file_path = "pcon-con.hbo.v7.allchannel.csv"
df = pd.read_csv(file_path)


In [64]:
def remove_outliers(data_frame, group):
    # Identify outliers in the PSD+AGG group using IQR method
    df_psd_agg = data_frame[data_frame['group'] == group]
    Q1 = df_psd_agg[['saps', 'sans', 'dose']].quantile(0.25)
    Q3 = df_psd_agg[['saps', 'sans', 'dose']].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier criteria
    outlier_mask = ((df_psd_agg[['saps', 'sans', 'dose']] < (Q1 - 1.5 * IQR)) | 
                    (df_psd_agg[['saps', 'sans', 'dose']] > (Q3 + 1.5 * IQR))).any(axis=1)

    # Remove outliers
    df_cleaned = df[~((df['group'] == 'PSD+AGG') & outlier_mask)]
    return df_cleaned

df_no_outliers = remove_outliers(df, 'PSD+AGG')
print(df_no_outliers)

                                                  path        volume    group  \
1    ../../data/patienter_lrv/0002P/stats.0002P.hbo...  0[corsi_har]  PSD+AGG   
2    ../../data/patienter_lrv/0003P/stats.0003P.hbo...  0[corsi_har]  PSD+AGG   
3    ../../data/patienter_lrv/0005P/stats.0005P.hbo...  0[corsi_har]  PSD+AGG   
4    ../../data/patienter_lrv/0008P/stats.0008P.hbo...  0[corsi_har]  PSD+AGG   
5    ../../data/patienter_lrv/0009P/stats.0009P.hbo...  0[corsi_har]  PSD+AGG   
..                                                 ...           ...      ...   
124  ../../data/friska_kontroller/2088K/stats.2088K...  0[corsi_har]       HC   
125  ../../data/friska_kontroller/2089K/stats.2089K...  0[corsi_har]       HC   
126  ../../data/friska_kontroller/2090K/stats.2090K...  0[corsi_har]       HC   
127  ../../data/friska_kontroller/2091K/stats.2091K...  0[corsi_har]       HC   
128  ../../data/friska_kontroller/2093K/stats.2093K...  0[corsi_har]       HC   

        id  sans  saps  dos

In [65]:
# Store results
results = []

# Perform Mann-Whitney U-test for each channel, adjusting for covariates
for channel in channels:
    # Drop missing values for robust analysis
    df_subset = df.dropna(subset=[group_col, channel] + covariates)

    # Encode the group as a binary variable (assumes two groups)
    unique_groups = df_subset[group_col].unique()
    if len(unique_groups) != 2:
        continue  # Skip if there are not exactly two groups

    group1, group2 = sorted(unique_groups)  # Ensure consistent group order
    data_group1 = df_subset[df_subset[group_col] == group1][channel]
    data_group2 = df_subset[df_subset[group_col] == group2][channel]

    # Mann-Whitney U-test
    stat, p_value = mannwhitneyu(data_group1, data_group2, alternative="two-sided")

    # Adjusting for covariates using a regression model
    formula = f"{channel} ~ C({group_col}, Treatment('{group1}')) + " + " + ".join(covariates)
    model = smf.ols(formula, data=df_subset).fit()
    group_term = f"C({group_col}, Treatment('{group1}'))[T.{group2}]"
    p_adj = model.pvalues.get(group_term, None)

    results.append({"Channel": channel, "Mann-Whitney p-value": p_value, "Adjusted p-value": p_adj})

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display significant results (p < 0.05)
significant_results = results_df[results_df["Adjusted p-value"] < 0.05]
tools.display_dataframe_to_user(name="Significant Channels", dataframe=significant_results)

Significant Channels


In [66]:
# Store interaction results for group interactions with covariates
group_interaction_results = []

# Check interactions between group and covariates for each channel
for channel in channels:
    # Drop missing values
    df_subset = df.dropna(subset=[group_col, channel] + covariates)

    # Encode the group variable properly
    unique_groups = df_subset[group_col].unique()
    if len(unique_groups) != 2:
        continue
    group1, group2 = sorted(unique_groups)  # Ensure consistent group order

    # Define the interaction model formula
    formula = f"{channel} ~ C({group_col}) * (sans + saps + dose)"
    model = smf.ols(formula, data=df_subset).fit()

    # Extract p-values for interaction terms
    interaction_terms = [f"C({group_col})[T.{group2}]:sans",
                         f"C({group_col})[T.{group2}]:saps",
                         f"C({group_col})[T.{group2}]:dose"]
    
    interaction_pvalues = {term: model.pvalues.get(term, None) for term in interaction_terms}

    # Store results
    group_interaction_results.append({"Channel": channel, **interaction_pvalues})

# Convert results to DataFrame
group_interaction_results_df = pd.DataFrame(group_interaction_results)

group_interaction_results_df.iloc[:, 1:] = group_interaction_results_df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# Display significant interactions (p < 0.05)
significant_mask = group_interaction_results_df.dropna().applymap(lambda x: x < 0.05 if isinstance(x, (int, float)) else False)
significant_channels_group = group_interaction_results_df[significant_mask.any(axis=1)]

# Display results to the user
tools.display_dataframe_to_user(name="Significant Group Interactions", dataframe=significant_channels_group)

Significant Group Interactions


/tmp/ipykernel_3297943/719355985.py:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  significant_mask = group_interaction_results_df.dropna().applymap(lambda x: x < 0.05 if isinstance(x, (int, float)) else False)
